# Organizando

In [1]:
!pip install nltk 
import nltk
nltk.download('punkt')


[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\muril\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Banco de dados de palavras

In [2]:
with open('Data/artigos.txt', 'r', encoding="utf8") as f:
    artigos = f.read()

In [3]:
with open('Data/br-sem-acentos.txt', 'r', encoding="utf8") as f:
    dicio = f.read()

In [4]:
texto_exemplo = 'Olá, tudo bem?'
tokens = texto_exemplo.split()
print(tokens)
# Token é uma sequência de caracteres, separados por um limitador,
# que pode ser um espaço em branco, pontuação ou quebra de linhas, por exemplo.

['Olá,', 'tudo', 'bem?']


In [5]:
palavras_separadas = nltk.tokenize.word_tokenize(texto_exemplo)
print(palavras_separadas)

['Olá', ',', 'tudo', 'bem', '?']


## Separador de palavras - tokenize

In [6]:
def separa_palavras(lista_tokens):
    lista_palavras = []
    for token in lista_tokens:
        if token.isalpha():
            lista_palavras.append(token)
    
    return lista_palavras

In [7]:
lista_tokens = nltk.tokenize.word_tokenize(artigos)
lista_tokens += nltk.tokenize.word_tokenize(dicio)
lista_palavras = separa_palavras(lista_tokens)
print(f'O numero de palavras é {len(lista_palavras)}')

O numero de palavras é 653351


## Normalizando todas as palavras

In [8]:
def normalizacao (lista_palavras):                          
    lista_normalizada = []
    for palavra in lista_palavras:
        lista_normalizada.append(palavra.lower())
    return lista_normalizada

In [9]:
lista_normalizada = normalizacao(lista_palavras)
len(lista_normalizada)

653351

## Funções do corretor
## insere, deleta, troca...

In [10]:
def insere_letras(fatias):
    novas_palavras = []
    letras = 'abcdefghijklmnopqrstuvwxyzàáâãèéêìíîòóôõùúûç'
    for E, D in fatias:
        for letra in letras:
            novas_palavras.append(E + letra + D)
    return novas_palavras


def deletando_letras (fatias):
    novas_palavras = []
    for E, D in fatias:
        novas_palavras.append(E + D[1:])
    return novas_palavras


def trocando_letras (fatias):
    letras = 'abcdefghijklmnopqrstuvwxyzàáâãèéêìíîòóôõùúûç'
    novas_palavras = []
    for E, D in fatias:
        for letra in letras:
            novas_palavras.append(E + letra + D[1:])
    return novas_palavras

def invertendo_letras(fatias):
    novas_palavras = []
    for E, D in fatias:
        if len(D) > 1:
            novas_palavras.append(E + D[1] + D[0] + D[2:])
    return novas_palavras

def gerador_palavras (palavra):
    
    fatias = []
    for i in range(len(palavra) + 1):               
        fatias.append((palavra[:i], palavra[i:]))   
    palavras_geradas = insere_letras(fatias)    
    palavras_geradas += deletando_letras(fatias)   
    palavras_geradas += trocando_letras(fatias) 
    palavras_geradas += invertendo_letras(fatias)
    
    return palavras_geradas

def gerador_turbinado(palavra):
    novas_palavras = [0]
    palavras_geradas = gerador_palavras(palavra)
    for palavra in palavras_geradas:
        novas_palavras += gerador_palavras(palavra)
    return novas_palavras


In [11]:
frequencia = nltk.FreqDist(lista_normalizada)
total_palavras = len(set(lista_normalizada))
frequencia.most_common(5)

def probabilidade (palavra_gerada):
    
    return frequencia[palavra_gerada]/total_palavras

## Lista com palavras corretas e erradas para uma avaliação do corretor

In [12]:
def cria_dados_teste(nome_arquivo):
    lista_palavras_teste = []
    f = open(nome_arquivo, "r", encoding="utf8")
    for linha in f:
        correta, errada = linha.split()
        lista_palavras_teste.append((correta, errada))
    f.close
    return lista_palavras_teste

In [13]:
lista_teste = cria_dados_teste("Data/palavras.txt")

## Corretor

In [14]:
def corretor (palavra):
    palavra2 = palavra
    candidatos = []
    palavras_geradas = gerador_palavras(palavra)
    candidatos = [palavra]
    for palavra in palavras_geradas:
        if palavra in vocabulario:
            candidatos.append(palavra)
        
    if len(candidatos) > 1:
        palavra_correta = max(candidatos, key = probabilidade)
                
    else:        
        palavras_geradas2 = gerador_palavras(palavra2)
        palavras_turbinado2 = gerador_turbinado(palavra2)
        todas_palavras2 = set(palavras_geradas2 + palavras_turbinado2)
        candidatos2 = [palavra2]
        for palavra2 in todas_palavras2:
            if palavra2 in vocabulario:
                candidatos2.append(palavra2)
           
        palavra_correta = max(candidatos2, key = probabilidade)

        
    return palavra_correta

## Avaliador

In [15]:
def avaliador (testes, vocabulario):
    numero_palavras = len(testes)
    acertou = 0
    desconhecida = 0
    for correta, errada in testes:
        palavra_corrigida = corretor(errada)
        desconhecida += (correta not in vocabulario)
        if palavra_corrigida == correta:
            acertou += 1

    taxa_acerto = round(acertou / numero_palavras * 100, 2)
    taxa_desconhecida = round(desconhecida / numero_palavras * 100, 2)
    print(f'{taxa_acerto}% de {numero_palavras}, desconhecidas sao: {taxa_desconhecida}%')

In [16]:
vocabulario = set(lista_normalizada)
len(vocabulario)

252528

In [17]:
avaliador(lista_teste, vocabulario)

90.32% de 186, desconhecidas sao: 0.0%


In [18]:
corretor('tomare')

'tomar'

In [21]:
def erradas (testes, vocabulario):
    for correta, errada in testes:
        palavra_corrigida = corretor(errada)
        if palavra_corrigida != correta:
            print(errada + '/' + correta + '/' + palavra_corrigida)

In [22]:
erradas(lista_teste, vocabulario)

eme/ele/em
noâ/nos/no
teb/ter/tem
âem/bem/em
serr/será/ser
eû/eu/e
jé/já/é
dms/das/dos
cava/cada/java
nossk/nossa/nosso
ìeu/eu/seu
qelay/ela/delay
deiìa/dia/deixa
eúaa/ela/essa
dĩaz/dia/da
asterístico/asterisco/asterístico
tomare/tomara/tomar
mindigo/mendigo/indigo
